In [ ]:
import numpy as np 
import pandas as pd
from datetime import datetime
from typing import List, Callable
import matplotlib.pyplot as plt 
import seaborn as sns 

# Librerías para el procesamiento de access logs 
import re 
from parse import parse 
from lars.apache import ApacheSource, COMBINED, ApacheWarning

# Manejo de advertencias del sistema, usada para capturar las líneas que no pueden parsearse por problemas de lars (ApacheWarning)
import warnings

# Configuración de estilo para las gráficas
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

# Ejecutar antes las celdas de `01_access_log_parsing` o elige una ubicación donde exista un archivo de access logs (Apache Combined Log Format) 
log_file_path = '../data/target/access_log_master.log'
log_prep_path = '../data/target/access_log_master.csv'

### Expresión General para Apache Combined Log Format

In [ ]:
LOG_PATTERN = re.compile(
  r'(?P<ip_client>\S+)\s+'      # IP del cliente 
  r'(?P<ident>\S+)\s+'          # ident 
  r'(?P<auth_user>\S+)\s+'      # auth_user
  r'\[(?P<timestamp>.+?)\]\s+'  # timestamp
  r'\"(?P<request>.*?)\"\s+'    # línea completa de request: método + url + protocolo
  r'(?P<status>\d{3})\s+'       # código de estado
  r'(?P<size>\S+)\s+'           # tamaño de respuesta (bytes)
  r'\"(?P<referer>.*?)\"\s+'    # referer
  r'\"(?P<user_agent>.*?)\"'    # user-agent
)

def parse_apache_logs(log_file_path, add_n_line:bool=False, del_request:bool=True):
  "Parsea logs de Apache extrayendo todos los campos solicitados"
  parsed_logs = []
  failed_lines = []
  
  with open(log_file_path, 'r', encoding='utf-8') as file:
    for line_num, line in enumerate(file, 1):
      line = line.strip()
      if not line:
        continue
      
      match_ = LOG_PATTERN.match(line)
      
      if match_:
        # Obtener todos los campos capturados
        log_data = match_.groupdict()
        
        # Separar request en método, URL y protocolo
        request_parts = log_data['request'].split()
        match len(request_parts):
          case 3:
            log_data['method'] = request_parts[0]
            log_data['url'] = request_parts[1]
            log_data['protocol'] = request_parts[2]
          case 2:
            log_data['method'] = request_parts[0]
            log_data['url'] = request_parts[1]
            log_data['protocol'] = 'UNKNOWN'
          case 1:
            log_data['method'] = 'UNKNOWN'
            log_data['url'] = request_parts[0]
            log_data['protocol'] = 'UNKNOWN'

        if add_n_line:
          # Añadir número de línea
          log_data['line_number'] = line_num
        
        if del_request:
          del log_data['request']
        
        parsed_logs.append(log_data)
      else:
        # Guardar línea fallida
        failed_lines.append({
          'line': line_num,
          'content': line[:100]
        })
  
  return parsed_logs, failed_lines

logs, errors = parse_apache_logs(log_file_path)

print(f"Cantidad de logs parseados: {len(logs)}")
print(f"Cantidad de logs fallidos:  {len(errors)}")

In [ ]:
# Inspección de logs parseados
logs[len(logs) - 1]

Crear una columna para etiquetar manualmente si un log es un ataque web o no

In [ ]:
df = pd.DataFrame(logs)

# Columna para representar si un log es un ataque: -1 si no se conoce, 1 si es ataque, o 0 si no es un ataque 
df['anomaly'] = -1

display(df.head())
display(df.tail())

In [ ]:
# transformar 'timestamp' a datetime (ajustando el formato de Apache) 
def parse_timestamps_vectorized(df):
  df['timestamp_clean'] = df['timestamp'].astype(str).str.strip('[]')
  
  df['timestamp_parsed'] = pd.to_datetime(
    df['timestamp_clean'],
    format='%d/%b/%Y:%H:%M:%S %z', 
    errors='coerce'
  )
  
  mask = df['timestamp_parsed'].isna()
  if mask.any():
    df.loc[mask, 'timestamp_parsed'] = pd.to_datetime(
      df.loc[mask, 'timestamp_clean'],
      format='%d/%b/%Y:%H:%M:%S',
      errors='coerce'
    )

  df['timestamp'] = df['timestamp_parsed']
  df = df.drop(['timestamp_parsed', 'timestamp_clean'], axis=1)
  
  return df 

df = parse_timestamps_vectorized(df)
# transformar 'size' a numérico (manejando valores no numéricos como '-')
df['size'] = pd.to_numeric(df['size'], errors='coerce').fillna(0)

In [ ]:
df['timestamp']

In [ ]:
print(f"Rango de Tiempo: {df['timestamp'].min()} -> {df['timestamp'].max()}")
print(f"Estadísticas de 'size': Min={ df['size'].min() }, Max={ df['size'].max() }, Mean={ df['size'].mean() }")

In [ ]:
# Mostrar las primeras K filas con los valores 'size' más altos
K = 10
display_cols = [
  
]
# Ordenar por tamaño de forma descendente y tomar las primeras K filas
top_K_largest = df.nlargest(K, 'size')
above_mean = df[df['size'] > df['size'].mean()]
print(f"Logs con tamaño de respuesta > promedio (promedio/mean={df['size'].mean():.2f}): {len(above_mean)} ({(len(above_mean)/len(df))*100:.2f}%)")
display(top_K_largest)

In [ ]:
plt.figure(figsize=(15, 6))

# Crear scatter plot con transparencia para manejar muchos puntos
plt.scatter(df['timestamp'], df['size'], alpha=0.6, s=10, color='royalblue')

plt.title('Tamaño de respuestas a lo largo del tiempo', fontsize=14, fontweight='bold')
plt.xlabel('Fecha y Hora', fontsize=12)
plt.ylabel('Tamaño (bytes)', fontsize=12)
plt.xticks(rotation=45)
plt.tight_layout()

plt.show()

In [ ]:
plt.figure(figsize=(15,6))

# agrupar por día 
df_daily = df.set_index('timestamp').resample('D').size()
# crear gráfica de barras para peticiones diarias 
bars = plt.bar(df_daily.index, df_daily.values, width=0.8, color='skyblue', edgecolor='navy', alpha=0.8)

plt.title('Número de peticiones por día', fontsize=14, fontweight='bold')
plt.xlabel('Fecha', fontsize=12)
plt.ylabel('Cantidad de peticiones', fontsize=12)
plt.xticks(rotation=45)

for bar in bars:
  height = bar.get_height()
  plt.text(bar.get_x() + bar.get_width()/2, height, f'{int(height)}', ha='center', va='bottom', fontsize=9, fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
important_columns = [
  'status',
  'method',
  'user_agent'
]

# Filtrar solo columnas existentes
existing_cols = [col for col in important_columns if col in df.columns]
n_cols = len(existing_cols)

# Configurar subplots
fig, axes = plt.subplots(n_cols, 1, figsize=(15, 5 * n_cols))
if n_cols == 1:
  axes = [axes]

# Crear gráficas para cada columna
for idx, col in enumerate(existing_cols):
  ax = axes[idx]
  
  # Obtener value counts (top 10 para evitar sobrecarga)
  value_counts = df[col].value_counts().head(10)
  
  # Para columnas con muchos valores únicos, mostrar solo top
  if len(value_counts) > 10:
    others_count = df[col].value_counts().iloc[10:].sum()
    if others_count > 0:
      value_counts['Otros'] = others_count
  
  # Crear gráfica de barras horizontal
  bars = ax.barh(range(len(value_counts)), value_counts.values, color=plt.cm.tab20c(range(len(value_counts))))
  
  ax.set_title(f'Distribución de {col}', fontsize=12, fontweight='bold')
  ax.set_xlabel('Frecuencia', fontsize=10)
  ax.set_yticks(range(len(value_counts)))
  ax.set_yticklabels(value_counts.index, fontsize=9)
  
  # Añadir etiquetas de valores
  total = value_counts.sum()
  for i, v in enumerate(value_counts.values):
    percentage = (v / total) * 100
    ax.text(v + max(value_counts.values) * 0.01, i, f'{v} ({percentage:.1f}%)', va='center', fontsize=9, fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Gráfica adicional: Códigos de estado HTTP (si existe la columna)
plt.figure(figsize=(12, 6))

# Agrupar códigos por categoría
df['status_category'] = df['status'].astype(str).str[0] + '00'

# Contar por categoría
status_counts = df['status_category'].value_counts()

# Crear gráfico de pastel
colors = ['#4CAF50', '#2196F3', '#FF9800', '#F44336', '#9C27B0']
wedges, texts, autotexts = plt.pie(status_counts.values, labels=status_counts.index, autopct='%1.1f%%', colors=colors, startangle=90)

# Mejorar etiquetas
for autotext in autotexts:
  autotext.set_color('white')
  autotext.set_fontweight('bold')

plt.title('Distribución de códigos de estado HTTP por categoría', fontsize=14, fontweight='bold')
plt.axis('equal')

# Añadir leyenda con códigos específicos más comunes
plt.figure(figsize=(10, 6))
top_status = df['status'].value_counts().head(10)
top_status.plot(kind='barh', color='steelblue')
plt.title('Top 10 códigos de estado HTTP más frecuentes', fontsize=12)
plt.xlabel('Frecuencia')
plt.tight_layout()
plt.show() 

### Búsqueda Automática de Logs Maliciosos a través del Parsing

Se puede utilizar la biblioteca de `lars` para identificar logs maliciosos, para ello se analizar todo el archivo de logs para intentar parsear cada línea y se separan los logs en 3 categorías: 
- `parsed_logs`: logs que fueron parseados con éxito y se extrajeron todas las características deseadas de esos logs (estas características las extrae `lars` pero no se tendrán en cuenta en la primera parte del análisis y parsing de logs)
- `conflicting_logs`: logs que fueron parseados con éxito pero existen características que no pueden ser extraídas (en el código siguiente se intenta manejar de forma automática estos tipos de logs y se definen valores `None` en caso de fallar al pedir una característica específica del log parseado por `lars`)
- `problematic_logs`: logs que no fueron parseados por `lars` debido a que presentan un patrón diferente al definido por `lars`, que es un patrón estándar para los logs de Apache (`ApacheSource`) con formato de logs combinado (`log_format=COMBINED` incluye User-Agent y Referrer)

In [ ]:
regex = '{ip_client} {ident} {auth_user} [{timestamp}] {request_http} {status:d} {size:d} {referrer} {agent}'

parsed_logs = []
conflicting_logs = {}
problematic_logs = {}

with open(log_file_path, 'r') as f:
  all_lines = f.readlines()

# Lista de campos de URL que se quieren extraer de la clase Row de lars
url_fields = [
  ('request_url_scheme', 'scheme'),
  ('request_url_netloc', 'netloc'),
  ('request_url_path_str', 'path_str'),
  ('request_url_params', 'params'),
  ('request_url_query_str', 'query_str'),
  ('request_url_fragment', 'fragment')
]

# Función para extraer campos de URL 
def extract_url_fields(url_obj):
  result = {}
  for field_name, attr_name in url_fields:
    try:
      value = getattr(url_obj, attr_name, None)
      result[field_name] = value if value else None
    except Exception:
      result[field_name] = None
  return result

with warnings.catch_warnings(record=True) as w:
  warnings.simplefilter("always", ApacheWarning)

  with open(log_file_path) as f:
    with ApacheSource(f, log_format=COMBINED) as source:
      for i, row in enumerate(source, 1):
        try:
          record = {
            "remote_host": row.remote_host,
            "ident": row.ident,
            "remote_user": row.remote_user,
            "time": row.time,
            "request_method": row.request.method if hasattr(row, 'request') and row.request else None,
            "request_protocol": row.request.protocol if hasattr(row, 'request') and row.request else None,
            "status": row.status,
            "size": row.size,
            "req_Referer": row.req_Referer,
            "req_User_agent": row.req_User_agent,
          }
          
          # Extraer campos de URL si existe
          if hasattr(row, 'request') and row.request and row.request.url:
            url_data = extract_url_fields(row.request.url)
            record.update(url_data)
          else:
            # Si no hay URL, establecer todos los campos como None
            record.update({field_name: None for field_name, _ in url_fields})
          
          line_content = all_lines[i-1].strip()
          parsed = parse(regex, line_content)
          if parsed:
            record['request_http'] = parsed.named['request_http']
          else: 
            print(f"Error?: Line: {line_content}")
          
          parsed_logs.append(record)
        except Exception as e:
          conflicting_logs[i] = {
            'error': str(e),
            'line_content': all_lines[i-1].strip() if i <= len(all_lines) else "not available"
          }
  
  for warning in w:
    if issubclass(warning.category, ApacheWarning):
      msg = str(warning.message)
      match = re.search(r'Line (\d+):', msg)
      if match:
        line_num = int(match.group(1))
        if line_num <= len(all_lines):
          problematic_logs[line_num] = {
            'line_content': all_lines[line_num - 1].strip(),
            'warning_message': msg,
            'category': warning.category.__name__,
          }

total_problematic_logs = len(problematic_logs)
total_conflicting_logs = len(conflicting_logs)
total_parsed_logs = len(parsed_logs)
total_logs = total_parsed_logs + total_problematic_logs + total_conflicting_logs

print(f"Total de logs procesados: {total_logs}")
print(f"Problematic Logs: {total_problematic_logs} ({(total_problematic_logs / total_logs * 100 if total_logs != 0 else 0):.2f}%)")
print(f"Conflicting Logs: {total_conflicting_logs} ({(total_conflicting_logs / total_logs * 100 if total_logs != 0 else 0):.2f}%)")
print(f"Parsed Logs: {total_parsed_logs} ({(total_parsed_logs / total_logs * 100 if total_logs != 0 else 0):.2f}%)")

In [ ]:
all_idx = list(problematic_logs.keys())
max_idx = max(all_idx)

line_problematics = [value['line_content'] for _,value in problematic_logs.items()]

for idx,log in problematic_logs.items():
  print(f"{idx:>{len(str(max_idx))}}) {log}")

In [ ]:
# Inspección de logs parseados
i = len(all_idx) - 1
display(logs[all_idx[i]-1])
display(df.iloc[all_idx[i]-1])

Presentan:
- Cadenas bytes en formato hexademical en las solicitudes HTTP (tanto en método, protocolo y url)
- No existe Referer, ni User-Agent
- Las pocas solicitudes HTTP (las que no están formadas por bytes en formato hexademical) no presentan método y tienen cadenas muy extrañas o tienen direcciones IP 

In [ ]:
from parse import parse 

pattern = '{ip_client} {ident} {auth_user} [{timestamp}] {request_http} {status:d} {size:d} "{referrer}" "{agent}"'

problematic_parsed_logs = []
for problematic in list(problematic_logs.items()):
  idx = problematic[0]
  line_content = problematic[1]['line_content']
  try: 
    parsed = parse(pattern, line_content.strip())
    problematic_parsed_logs.append(parsed.named)
  except Exception as e:
    print(f"Line {idx}: {line_content}")
    print(f"Error: {e}")

df_problematic = pd.DataFrame(problematic_parsed_logs)
status_distribution = df_problematic['status'].value_counts().sort_index()
display(df_problematic.head(3))
display(df_problematic.shape[0])
display(status_distribution)

**Análisis de Distribución de Código de Estado HTTP**:
- `301`: El recurso solicitado tiene una nueva URL permanente 
- `400`: El servidor no puede entender la solicitud debido a una sintaxis inválida 
- `408`: El servidor esperó demasiado tiempo a que el cliente enviara la solicitud completa y cerró la conexión inactiva
- `499`: El cliente cerro la solicitud antes de que el servidor pudiera responder

Para códigos 400, se va revisar la dirección IP de origen y el agente de usuario $\to$ Un alto volumen de errores 400 provenientes de unas pocas IPs o de agentes de usuario inusuales (como herramientas de escaneo) puede indicar actividad maliciosa automatizada. 

Para códigos 408 y 499: Estos códigos suelen ser síntomas de que algo en el servidor no funciona de manera óptima $\to$ Un aumento repentino suele estar relacionado con una alta carga en el servidor, tiempos de respuesta lentos en la aplicación o problemas de red

In [ ]:
# Filtrar logs con los códigos de estado de interés
status_codes_to_check = [400, 408, 499]
filtered_logs = df_problematic[df_problematic['status'].isin(status_codes_to_check)]

# Agrupar por IP para identificar posibles orígenes problemáticos
ip_analysis = filtered_logs.groupby(['status', 'ip_client']).size().reset_index(name='count')
print(ip_analysis.sort_values(by='count', ascending=False).head(20))  # Ver las 20 IPs más activas

# Agrupar por agente de usuario
agent_analysis = filtered_logs.groupby(['status', 'agent']).size().reset_index(name='count')
print(agent_analysis.sort_values(by='count', ascending=False).head(10))  # Ver los 10 agentes más comunes

Estos resultados muestran una **actividad maliciosa**: patrón de cientos de errores 400, concentrados en pocas IPs, cada una con rangos muy diferentes, y con agente vacío, no es un comportamiento normal de un usuario o buscador. 

In [ ]:
adjusted_idx = [idx-1 for idx in all_idx] 
df.loc[adjusted_idx, 'anomaly'] = 1
display(df['anomaly'].head())
display(df[ df['anomaly'] == 1 ].head(10))

### Análisis de `user-agent` sospechosos

In [ ]:
df_user_agent = pd.DataFrame(df['user_agent'].value_counts())
display(df_user_agent.head(5))
display(df_user_agent.columns)

In [ ]:
def is_bot(user_agent:str) -> bool:
  "Detecta si un user-agent pertenece a un bot/crawler/spider"
  if not user_agent:
    return False 
  user_agent = user_agent.lower()
  
  # Nombres de bots conocidos
  known_bots = {
    'googlebot', 
    'bingbot', 
    'yandexbot', 
    'applebot',
    'duckduckbot', 
    'baiduspider', 
    'sogou', 
    'bytespider',
    'amazonbot', 
    'gptbot', 
    'chatgpt-user', 
    'oai-searchbot',
    'claudebot', 
    'google-cloudvertexbot', 
    'google-extended',
    'perplexitybot', 
    'meta-externalagent', 
    'meta-webindexer',
    'tiktokspider', 
    'openai.com-bot', 
    'google.bot',
    # Poco comunes pero encontrados en el archivo de access log
    'thinkbot', 
    'petalbot'
    # No es bot pero se asumirá que sí debido a que los comportamientos no son permitidos
    'securitytxtresearch'
    #'SecurityTxtResearch'
  }

  # Verificar nombres de bots conocidos
  for bot in known_bots:
    if bot in user_agent:
      return True

  # Patrón: "dominio.com-bot" o "dominio.bot"
  import re
  pattern = r'[a-z0-9.-]+\.(?:com|org|net|io)[-.]bot'
  if re.search(pattern, user_agent):
    return True

  return False

In [ ]:
df['is_bot'] = df['user_agent'].apply(is_bot)

In [ ]:
display(df.head())
display(df['is_bot'].value_counts())

In [ ]:
# Crear máscara para los bots
bot_mask = df['is_bot'] == True
# Modificar la columna 'attack' para los bots 
df.loc[bot_mask, 'anomaly'] = 1 

### Análisis de Métodos 

In [ ]:
df_filtered = df[(df['method'] != 'GET') & (df['method'] != 'POST')]
display(df_filtered)

In [ ]:
method_counts = df_filtered['method'].value_counts()
percentage_series = (method_counts / len(df_filtered) * 100).round(2)
summary_df = pd.DataFrame({
  'count': method_counts, 
  'percentage': percentage_series
})
display(summary_df)

In [ ]:
df_del_method = df_filtered[df_filtered['method'] == 'DELETE']
display(df_del_method)

In [ ]:
df_trace_method = df_filtered[df_filtered['method'] == 'TRACE']
display(df_trace_method)

In [ ]:
del_method_mask = df['method'] == 'DELETE'
df.loc[del_method_mask, 'anomaly'] = 1
trace_method_mask = df['method'] == 'TRACE'
df.loc[trace_method_mask, 'anomaly'] = 1

### Análisis de IPs de Clientes

In [ ]:
df_filtered = df[df['ip_client'] == '::1']
display(df_filtered)

In [ ]:
df_filtered['method'].value_counts()

In [ ]:
df_filtered['status'].value_counts()

In [ ]:
df_filtered['url'].value_counts()

In [ ]:
mask = df['ip_client'] == '::1'
df.loc[mask, 'anomaly'] = 0      # El resultado de investigar los logs con estos IP llego a ser que son logs no maliciosos

### Extracción y Codificación de Características

In [ ]:
def count_sql_words(url):
  "Cuenta palabras relacionadas con SQL Injection" 
  sql_words = [
    r'SELECT', 
    r'FROM', 
    r'WHERE', 
    r'DELETE', 
    r'DROP', 
    r'CREATE', 
    r'TABLE', 
    r'LIKE', 
    r'UNION', 
    r'INSERT', 
    r'UPDATE', 
    r'ALTER',
    r'INTO', 
    r'VALUES', 
    r'SET', 
    r'JOIN', 
    r'GRANT', 
    r'REVOKE'
  ]
  pattern = re.compile('|'.join(sql_words), re.IGNORECASE)
  matches = pattern.findall(str(url))
  return len(matches)

def count_xss_words(url):
  "Cuenta palabras relacionadas con Cross-Site Scripting (XSS)"
  xss_words = [
    r'script', 
    r'alert', 
    r'javascript', 
    r'onerror', 
    r'onload', 
    r'onunload', 
    r'prompt', 
    r'confirm', 
    r'eval', 
    r'expression',
    r'function\(', 
    r'xmlhttprequest', 
    r'xhr', 
    r'window\.', 
    r'document\.', 
    r'iframe', 
    r'src=', 
    r'cookie', 
    r'document\.cookie',
    r'set-cookie', 
    r'click', 
    r'mouseover'
  ]
  pattern = re.compile('|'.join(xss_words), re.IGNORECASE)
  matches = pattern.findall(str(url))
  return len(matches)

def count_command_words(url): 
  "Cuenta palabras relacionadas con ejecución de comandos"
  command_words = [
    r'cmd', 
    r'dir', 
    r'shell', 
    r'exec', 
    r'cat', 
    r'etc', 
    r'tmp',
    r'bin', 
    r'bash', 
    r'sh', 
    r'python', 
    r'perl', 
    r'ruby', 
    r'php',
    r'\.exe', 
    r'\.php', 
    r'\.js', 
    r'\.py', 
    r'\.pl', 
    r'\.rb',
    r'system\(', 
    r'popen\(', 
    r'proc_open\(', 
    r'passthru\('
  ]
  pattern = re.compile('|'.join(command_words), re.IGNORECASE)
  matches = pattern.findall(str(url))
  return len(matches)

def count_auth_words(url):
  "Cuenta palabras relacionadas con autentificación"
  auth_words = [
    r'admin', 
    r'administrator', 
    r'password', 
    r'login', 
    r'pwd',
    r'credential', 
    r'user', 
    r'username', 
    r'passwd', 
    r'secret',
    r'token', 
    r'session', 
    r'auth', 
    r'authentication', 
    r'key'
  ]
  pattern = re.compile('|'.join(auth_words), re.IGNORECASE)
  matches = pattern.findall(str(url))
  return len(matches)

def count_error_words(url):
  "Cuenta palabras relacionadas con errores"
  e_words = [
    r'error', 
    r'errorMsg', 
    r'errorID', 
    r'incorrect', 
    r'fail',
    r'failed', 
    r'failure', 
    r'exception', 
    r'stack',
    r'trace',
    r'debug', 
    r'warning', 
    r'fatal', 
    r'crash',
    r'invalid'
  ]
  pattern = re.compile('|'.join(e_words), re.IGNORECASE)
  matches = pattern.findall(str(url))
  return len(matches)

def count_malware_words(url):
  "Cuenta palabras relacionadas con malware"
  malware_words = [
    r'malware', 
    r'ransomware', 
    r'phishing', 
    r'exploit', 
    r'virus',
    r'trojan', 
    r'backdoor', 
    r'spyware', 
    r'rootkit', 
    r'worm',
    r'adware', 
    r'keylogger', 
    r'botnet', 
    r'payload', 
    r'inject',
    r'injected', 
    r'hacker', 
    r'attack', 
    r'exploit', 
    r'breach'
  ]
  pattern = re.compile('|'.join(malware_words), re.IGNORECASE)
  matches = pattern.findall(str(url))
  return len(matches)

def count_danger_characters(url):
  "Cuenta caracteres potencialmente peligrosos"
  characters = [
    r"'", 
    r"--", 
    r";", 
    r"\\", 
    r"\"", 
    r"<", 
    r">", 
    r"(", 
    r")", 
    r"&", 
    r"|"
  ]
  count = 0
  url_str = str(url)
  for c in characters:
    count += url_str.count(c)
  return count

def count_obfuscation_code_words(url):
  "Cuenta técnicas de ofuscación de código"
  obfuscation_words = [
    r'encode', 
    r'decode', 
    r'base64', 
    r'hex', 
    r'urlencode',
    r'urldecode', 
    r'escape', 
    r'unescape', 
    r'obfuscate',
    r'xor', 
    r'rot13', 
    r'chr\(',
    r'char\(', 
    r'fromCharCode',
    r'eval\('
  ]
  pattern = re.compile('|'.join(obfuscation_words), re.IGNORECASE)
  matches = pattern.findall(str(url))
  return len(matches)

def count_dir_words(url):
  "Cuenta referencias a directorios sensibles"
  dir_words = [
    r'\.\./', 
    r'\.\.\\', 
    r'/etc/', 
    r'/bin/', 
    r'/tmp/', 
    r'/var/',
    r'/home/', 
    r'/root/', 
    r'proc/', 
    r'dev/', 
    r'boot/', 
    r'usr/', 
    r'lib/', 
    r'sbin/'
  ]
  pattern = re.compile('|'.join(dir_words), re.IGNORECASE)
  matches = pattern.findall(str(url))
  return len(matches)

def count_dot(url):
  "Cuenta la cantidad de puntos en la URL"
  url = str(url)
  return url.count('.')

def count_http(url): 
  "Cuenta las ocurrencias de http en la URL"
  url = str(url)
  return url.count('http')

def count_percentage_symbol(url):
  "Cuenta los signos de porcentaje"
  url = str(url)
  return url.count('%')

def count_question_symbol(url):
  "Cuenta los signos de interrogación"
  url = str(url)
  return url.count('?')

def count_hyphen(url):
  "Cuenta guiones (-) en la URL"
  url = str(url)
  return url.count('-')

def count_equal(url):
  "Cuenta signos igual (=)"
  url = str(url)
  return url.count('=')

def url_length(url):
  "Retorna la longitud total de la URL"
  return len(str(url))

def digit_count(url):
  "Cuenta la cantidad de dígitos numéricos en la URL"
  digits = 0
  for i in url:
    if i.isnumeric():
      digits = digits + 1
  return digits

def letter_count(url):
  "Cuenta la cantidad de letras en la URL"
  letters = 0
  for i in url:
    if i.isalpha():
      letters += 1
  return letters

def count_special_characters(url):
  "Cuenta caracteres especiales (no alfanuméricos) usando regex"
  special_characters = re.sub(r'[a-zA-Z0-9\s]', '', url)
  count = len(special_characters)
  return count

def is_encoded(url):
  """Detecta si la URL está codificada (presencia de %)

  Returns:
    int: Retorna 1 si es verdadero (URL está codificada) y 0 si no
  """
  return int('%' in url.lower())

def unusual_character_ratio(url):
  "Calcula la proporción de caracteres inusuales (no alfanuméricos, guiones, puntos o guiones bajos) respecto a la longitud total"
  total_characters = len(url)
  unusual_characters = re.sub(r'[a-zA-Z0-9\s\-._]', '', url)
  unusual_count = len(unusual_characters)
  ratio = unusual_count / total_characters if total_characters > 0 else 0
  return ratio

In [ ]:
extract_features_functions: List[Callable] = [
  count_sql_words,
  count_xss_words, 
  count_command_words, 
  count_auth_words, 
  count_error_words,
  count_malware_words,
  count_danger_characters,
  count_obfuscation_code_words,
  count_dir_words,
  count_dot,
  count_http,
  count_percentage_symbol,
  count_question_symbol,
  count_hyphen,
  count_equal,
  url_length,
  digit_count,
  letter_count,
  count_special_characters,
  is_encoded,
  unusual_character_ratio
]

def safe_str_conversion(value):
  if pd.isna(value):
    return ''
  if isinstance(value, str):
    return value 
  # para cualquier otro tipo (int, float, etc), convertir a str 
  return str(value)
df['url'] = df['url'].apply(safe_str_conversion)

request_columns_name = []
for func in extract_features_functions:
  feat_name = f"url__{func.__name__}"
  df[feat_name] = df['url'].apply(func)
  request_columns_name.append(feat_name)

display(df.head())

In [ ]:
# codificar ip-client
import ipaddress

def encode_ip(ip):
  if pd.isna(ip):
    return 2  
  ip_str = str(ip).strip()
  
  if ip_str == '::1':
    return 1  # localhost
  
  return 0    # other

df['encode_ip'] = df['ip_client'].apply(encode_ip)
display(df['encode_ip'].value_counts())


In [ ]:
def encoder_log(df:pd.DataFrame, col_name):
  if col_name not in df.columns:
    raise Exception("La columna no existe en el DataFrame")
  serie = df[col_name].copy()
  serie = serie.astype(str).str.strip()
  encoder_result = np.where(
    serie.isin(['-', '', 'nan', 'None', 'null']) | (serie == 'nan'),
    0,  # ausente/no aplicable
    1   # presente
  )
  return encoder_result

df['ident_encoded'] = encoder_log(df, 'ident')
df['user_encoded']  = encoder_log(df, 'auth_user')

In [ ]:
def is_valid_method(method):
  "Verifica si el método parece un método HTTP válido"
  if pd.isna(method):
    return False
  # Métodos HTTP válidos según RFC + 'UNKNOWN'
  valid_pattern = r'^[A-Z_-]{1,20}$'
  return bool(re.match(valid_pattern, str(method)))

# Identificar valores inválidos
invalid_mask = ~df['method'].apply(is_valid_method)
invalid_methods = df.loc[invalid_mask, 'method'].unique()

print(f"Valores inválidos/ruido encontrados: {len(invalid_methods)}")
if len(invalid_methods) > 0:
  print("Ejemplos:", invalid_methods[:5])

# Marcar como 'INVALID'
df['method'] = df['method'].apply(lambda x: 'INVALID' if not is_valid_method(x) else x)

In [ ]:
df['method'].value_counts()

In [ ]:
target_methods = ['DELETE', 'TRACE'] # 'PUT', 'REQMOD'

# Filtrar el DataFrame para obtener solo estas solicitudes
df_target = df[df['method'].isin(target_methods)].copy()

# Ordenar por método para mejor visualización
df_target = df_target.sort_values('method')

# Mostrar todas las filas con columnas relevantes
cols_to_display = ['method', 'url', 'ip_client', 'user_agent']
# Ajustar según las columnas disponibles en tu DataFrame
available_cols = [col for col in cols_to_display if col in df_target.columns]

print(df_target[available_cols].to_string(index=False))

# Si hay muchas columnas, podemos mostrar un resumen compacto
for method in target_methods:
  method_data = df_target[df_target['method'] == method]
  if len(method_data) > 0:
    print(f"\n--- {method} ({len(method_data)} solicitudes) ---")
    # Mostrar URL únicas y IPs asociadas
    print(f"URLs únicas: {method_data['url'].nunique()}")
    for url in method_data['url'].unique():
      ips = method_data[method_data['url'] == url]['ip_client'].unique()
      print(f"  - {url[:80]}... (IPs: {', '.join(ips)})")

In [ ]:
target_methods = ['DELETE', 'PUT', 'TRACE', 'REQMOD', 'INVALID']

# Crear máscara para identificar las filas con estos métodos
mask = df['method'].isin(target_methods)
# Establecer attack = 1 para los métodos objetivo
df.loc[mask, 'anomaly'] = 1

In [ ]:
from sklearn.preprocessing._label import LabelEncoder

status_series = df['status_category'].copy()
status_series = status_series.fillna('UNKNOWN')
status_encoder = LabelEncoder()
df['status_category_encoded'] = status_encoder.fit_transform(status_series)

category_mapping = dict(zip(status_encoder.classes_, status_encoder.transform(status_encoder.classes_)))
display(category_mapping)

In [ ]:
method_series = df['method'].copy()
method_series = method_series.fillna('UNKNOWN')
method_encoder = LabelEncoder()
df['method_encoded'] = method_encoder.fit_transform(method_series)

status_mapping = dict(zip(method_encoder.classes_, method_encoder.transform(method_encoder.classes_)))
display(status_mapping)

In [ ]:
df.columns

In [ ]:
y_label_column = 'anomaly'
columns_selected = [
  'size', 
  'method_encoded', 
  'is_bot', 
  'encode_ip', 
  'status_category_encoded', 
  'ident_encoded',
  'user_encoded'
] + request_columns_name + [y_label_column]

In [ ]:
df_selected = df[columns_selected]

### Análisis y Etiquetado Manual de Datos Restantes

In [ ]:
#display(df.head(5))
display(df['anomaly'].value_counts())

In [ ]:
labeled_data = df[df['anomaly'] != -1]
unlabeled_data = df[df['anomaly'] == -1]
display(unlabeled_data.head(5))

In [ ]:
import pandas as pd
from IPython.display import clear_output

def manual_labeling(unlabeled_data:pd.DataFrame, columns_selected:List[str], _max:int=20):
  for idx in unlabeled_data.index[:_max]:
    clear_output(wait=True) 
    data = unlabeled_data.loc[idx] 
    
    # Mostrar información del log no etiquetado
    max_len_columns_name = max([len(col_name) for col_name in columns_selected])
    for col_name in columns_selected:
      print(f"{col_name:>{max_len_columns_name}}: {data[col_name]}")
    
    option = input("\n0=Normal, 1=Anomalía, s=Saltar, q=Salir: ").strip()

    match option:
      case '0':
        df.at[idx, 'anomaly'] = 0
      case '1':
        df.at[idx, 'anomaly'] = 1 
      case 'q':
        break 

  return unlabeled_data

df_manual = manual_labeling(
  unlabeled_data, 
  columns_selected=[
    'ip_client', 
    'ident',
    'auth_user',
    'status', 
    'size', 
    'user_agent', 
    'method', 
    'url', 
    'protocol',
  ], 
  _max=100
)

In [ ]:
display(df_manual.head(10))
idx_update = df_manual[df_manual['anomaly'] != -1].index
df.loc[idx_update, 'anomaly'] = df_manual.loc[idx_update, 'anomaly']

### Guardar el Dataframe

In [ ]:
df.to_csv(log_prep_path, index=False)